In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import metrics

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D


import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import sklearn.model_selection
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

#word2vec
from gensim.models import Word2Vec  
import gensim
import string

[nltk_data] Downloading package wordnet to /home/typedoor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
PATH = '/home/typedoor/PLBRS/Data/'

In [3]:
sp = pd.read_csv(PATH + 'south_park.csv')

In [7]:
sp_clean = sp[(sp.Character == 'Cartman') | (sp.Character == 'Kyle') | (sp.Character == 'Stan') | (sp.Character == 'Butters') | (sp.Character == 'Randy')].reset_index(drop = True)
sp_clean.shape  

(29622, 4)

In [8]:
sp_copy = sp_clean.copy()[['Character', 'Line']]
sp_copy.Line = sp_copy.Line.apply(sent_tokenize)
sp_copy.Line[0]

['You guys, you guys!', 'Chef is going away.']

In [9]:
def break_up_to_sentences(df):
    result_df = pd.DataFrame(columns = ['Character', 'Line'])
    
    for index, row in df.iterrows():
        if index %1000 == 0:
                print(index)
        
        for sentence in row.Line:

            addition = {'Character': row.Character,'Line': sentence }
            result_df = result_df.append(addition, ignore_index = True)
    
    return result_df
full_df = break_up_to_sentences(sp_copy) 
full_df

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


,Character,Line
0,Stan,"You guys, you guys!"
1,Stan,Chef is going away.
2,Kyle,Going away?
3,Kyle,For how long?
4,Stan,Forever.
5,Stan,"Chef said he's been bored, so he joining a gro..."
6,Cartman,I'm gonna miss him.
7,Cartman,I'm gonna miss Chef and I...and I don't know h...
8,Stan,"Dude, how are we gonna go on?"
9,Stan,Chef was our fuh...f-ffriend.


In [10]:
full_df.groupby('Character').count().Line[sp.groupby('Character').count().Line > 2000]

Character
Butters     5041
Cartman    21122
Kyle       11517
Randy       5284
Stan       11847
Name: Line, dtype: int64

In [26]:
working_df = full_df[(full_df.Character == 'Cartman') | (full_df.Character == 'Kyle') | (full_df.Character == 'Stan')].reset_index(drop = True)
indices = working_df[(working_df.Line == '!') | (working_df.Line == '?')].index
final_df = working_df.drop(index = indices).reset_index(drop = True)


In [23]:
final_df

,index,Character,Line
0,0,Stan,"You guys, you guys!"
1,1,Stan,Chef is going away.
2,2,Kyle,Going away?
3,3,Kyle,For how long?
4,4,Stan,Forever.
5,5,Stan,"Chef said he's been bored, so he joining a gro..."
6,6,Cartman,I'm gonna miss him.
7,7,Cartman,I'm gonna miss Chef and I...and I don't know h...
8,8,Stan,"Dude, how are we gonna go on?"
9,9,Stan,Chef was our fuh...f-ffriend.


In [27]:
final_df.to_csv(PATH + 'south_park_sentence_by_sentence.csv', index = False)

In [28]:
simpsons = pd.read_csv(PATH+ 'simpsons.csv')
simpsons.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [33]:
simpsons.groupby('raw_character_text').count().spoken_words[simpsons.groupby('raw_character_text').count().spoken_words > 2000]

raw_character_text
Bart Simpson           12995
C. Montgomery Burns     3077
Homer Simpson          27850
Lisa Simpson           10756
Marge Simpson          13172
Moe Szyslak             2808
Ned Flanders            2056
Seymour Skinner         2385
Name: spoken_words, dtype: int64

In [55]:
simpsons.rename(columns = {'raw_character_text': 'Character','spoken_words' : 'Line'}, inplace = True)
simpsons_family = simpsons[(simpsons.Character == 'Bart Simpson') | (simpsons.Character == 'Marge Simpson') | (simpsons.Character == 'Homer Simpson') | (simpsons.Character == 'Lisa Simpson')].reset_index(drop = True)
simpsons_family.dropna(inplace = True)
simpsons_family.head(25)

,Character,Line
0,Lisa Simpson,Where's Mr. Bergstrom?
1,Lisa Simpson,That life is worth living.
2,Bart Simpson,Victory party under the slide!
3,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
4,Lisa Simpson,Do you know where I could find him?
5,Lisa Simpson,"The train, how like him... traditional, yet en..."
6,Lisa Simpson,"I see he touched you, too."
7,Bart Simpson,"Hey, thanks for your vote, man."
8,Bart Simpson,"Well, you got that right. Thanks for your vote..."
9,Bart Simpson,"Well, don't sweat it. Just so long as a couple..."


In [57]:
set(simpsons_family.Character)

{'Bart Simpson', 'Homer Simpson', 'Lisa Simpson', 'Marge Simpson'}

In [60]:
simpsons_family.Line = simpsons_family.Line.apply(sent_tokenize)
simpsons_full = break_up_to_sentences(simpsons_family)
simpsons_full.head(25)

0
1000
2000
3000
4000
5000
6000
7000
8000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
59000
60000
61000
62000
63000
65000
66000
67000
68000
69000


,Character,Line
0,Lisa Simpson,Where's Mr. Bergstrom?
1,Lisa Simpson,That life is worth living.
2,Bart Simpson,Victory party under the slide!
3,Lisa Simpson,Mr. Bergstrom!
4,Lisa Simpson,Mr. Bergstrom!
5,Lisa Simpson,Do you know where I could find him?
6,Lisa Simpson,"The train, how like him... traditional, yet en..."
7,Lisa Simpson,"I see he touched you, too."
8,Bart Simpson,"Hey, thanks for your vote, man."
9,Bart Simpson,"Well, you got that right."


In [63]:
simpsons_full.shape

(101914, 2)

In [66]:
indices = simpsons_full[(simpsons_full.Line == '!') | (simpsons_full.Line == '?')].index
simpsons_full = simpsons_full.drop(index = indices).reset_index(drop = True)
simpsons_full.shape

(101167, 2)

In [68]:
simpsons_full.to_csv(PATH+'simpson_family_sentence_by_sentence.csv', index = False)

In [70]:
simpsons_full.groupby('Character').count()

,Line
Character,
Bart Simpson,19486
Homer Simpson,46333
Lisa Simpson,15873
Marge Simpson,19475


**Trying to clssify simpsons**


In [75]:
y = simpsons_full.Character.values
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
text_tf= tf.fit_transform(simpsons_full['Line'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_tf, y, test_size=0.3, random_state=123)

In [80]:
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
confusion = metrics.confusion_matrix(y_test, predicted, labels = ['Bart Simpson', 'Lisa Simpson','Homer Simpson','Marge Simpson'])
cm = metrics.classification_report(y_test, predicted,labels = ['Bart Simpson', 'Lisa Simpson','Homer Simpson','Marge Simpson'])
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))
print(confusion)
print(cm)

MultinomialNB Accuracy: 0.4947448189515996
[[  565   134  5161    65]
 [  185   318  4135    70]
 [  140    72 13567   146]
 [   57    37  5133   566]]
               precision    recall  f1-score   support

 Bart Simpson       0.60      0.10      0.16      5925
 Lisa Simpson       0.57      0.07      0.12      4708
Homer Simpson       0.48      0.97      0.65     13925
Marge Simpson       0.67      0.10      0.17      5793

    micro avg       0.49      0.49      0.49     30351
    macro avg       0.58      0.31      0.28     30351
 weighted avg       0.55      0.49      0.38     30351



In [83]:
clf = RandomForestClassifier(n_jobs = -1, n_estimators = 7500, bootstrap = True, max_depth = 2,class_weight= 'balanced_subsample').fit(X_train,y_train)
predicted= clf.predict(X_test)
confusion = metrics.confusion_matrix(y_test, predicted, labels = ['Bart Simpson', 'Lisa Simpson','Homer Simpson','Marge Simpson'])
cm = metrics.classification_report(y_test, predicted,labels = ['Bart Simpson', 'Lisa Simpson','Homer Simpson','Marge Simpson'])
print("RF Accuracy:",metrics.accuracy_score(y_test, predicted))
print(confusion)
print(cm)

Random Forest Accuracy: 0.3982735329972653
RF Accuracy: 0.3982735329972653
[[2341 1019 1332 1233]
 [1089 1577  909 1133]
 [3567 1639 5288 3431]
 [ 887  780 1244 2882]]
               precision    recall  f1-score   support

 Bart Simpson       0.30      0.40      0.34      5925
 Lisa Simpson       0.31      0.33      0.32      4708
Homer Simpson       0.60      0.38      0.47     13925
Marge Simpson       0.33      0.50      0.40      5793

    micro avg       0.40      0.40      0.40     30351
    macro avg       0.39      0.40      0.38     30351
 weighted avg       0.45      0.40      0.41     30351

